In [1]:
import os
import json
import torch
import pickle
import json
import bertviz
import uuid
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import MultipleLocator
from collections import Counter
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.core.display import display, HTML, Javascript
from bertviz.util import format_special_chars, format_attention, num_layers, num_heads
LAYER_NUM = 28
HEAD_NUM = 16
HEAD_DIM = 256
HIDDEN_DIM = HEAD_NUM * HEAD_DIM
torch.set_default_device("cuda:1")

In [2]:
LAYER_NUM = 28
HEAD_NUM = 16
HEAD_DIM = 256
HIDDEN_DIM = HEAD_NUM * HEAD_DIM
torch.set_default_device("cuda:1")

In [3]:
def transfer_output(model_output):
    all_pos_layer_input = []
    all_pos_attn_output = []
    all_pos_residual_output = []
    all_pos_ffn_output = []
    all_pos_layer_output = []
    all_last_attn_subvalues = []
    all_pos_coefficient_scores = []
    all_attn_scores = []
    for layer_i in range(LAYER_NUM):
        cur_layer_input = model_output[layer_i][0]
        cur_attn_output = model_output[layer_i][1]
        cur_residual_output = model_output[layer_i][2]
        cur_ffn_output = model_output[layer_i][3]
        cur_layer_output = model_output[layer_i][4]
        cur_last_attn_subvalues = model_output[layer_i][5]
        cur_coefficient_scores = model_output[layer_i][6]
        cur_attn_weights = model_output[layer_i][7]
        all_pos_layer_input.append(cur_layer_input[0].tolist())
        all_pos_attn_output.append(cur_attn_output[0].tolist())
        all_pos_residual_output.append(cur_residual_output[0].tolist())
        all_pos_ffn_output.append(cur_ffn_output[0].tolist())
        all_pos_layer_output.append(cur_layer_output[0].tolist())
        all_last_attn_subvalues.append(cur_last_attn_subvalues[0].tolist())
        all_pos_coefficient_scores.append(cur_coefficient_scores[0].tolist())
        all_attn_scores.append(cur_attn_weights)
    return all_pos_layer_input, all_pos_attn_output, all_pos_residual_output, all_pos_ffn_output, \
           all_pos_layer_output, all_last_attn_subvalues, all_pos_coefficient_scores, all_attn_scores
def get_bsvalues(vector, model, final_var):
    E = torch.mean(vector, -1)
    vector_ln = (vector-E.unsqueeze(-1))/final_var * model.transformer.ln_f.weight.data
    vector_bsvalues = model.lm_head(vector_ln).data
    return vector_bsvalues
def get_prob(vector):
    prob = torch.nn.Softmax(-1)(vector)
    return prob
def get_fc2_params(model, layer_num):
    return model.transformer.h[layer_num].mlp.fc_out.weight.data
def transfer_l(l):
    new_x, new_y = [], []
    for x in l:
        new_x.append(x[0])
        new_y.append(x[1])
    return new_x, new_y
def plt_bar(x, y, yname="log increase"):
    x_major_locator=MultipleLocator(1)
    plt.figure(figsize=(8, 3))
    ax=plt.gca()
    ax.xaxis.set_major_locator(x_major_locator)
    plt_x = [a/2 for a in x]
    plt.xlim(-0.5, plt_x[-1]+0.49)
    x_attn, y_attn, x_ffn, y_ffn = [], [], [], []
    for i in range(len(x)):
        if i%2 == 0:
            x_attn.append(x[i]/2)
            y_attn.append(y[i])
        else:
            x_ffn.append(x[i]/2)
            y_ffn.append(y[i])
    plt.bar(x_attn, y_attn, color="darksalmon", label="attention layers")
    plt.bar(x_ffn, y_ffn, color="lightseagreen", label="FFN layers")
    plt.xlabel("layer")
    plt.ylabel(yname)
    plt.legend()
    plt.show()
def plt_heatmap(data):
    xLabel = range(len(data[0]))
    yLabel = range(len(data))
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111)
    ax.set_xticks(range(len(xLabel)))
    ax.set_xticklabels(xLabel)
    ax.set_yticks(range(len(yLabel)))
    ax.set_yticklabels(yLabel)
    im = ax.imshow(data, cmap=plt.cm.hot_r)
    #plt.colorbar(im)
    plt.title("attn head log increase heatmap")
    plt.show()

In [ ]:
modelname = "model_gptj"
tokenizer = AutoTokenizer.from_pretrained(modelname)
model = AutoModelForCausalLM.from_pretrained(modelname)
model.eval()
model.cuda()

In [7]:
test_sentence = "Tim Duncan plays the sport of"
indexed_tokens = tokenizer.encode(test_sentence)
tokens = [tokenizer.decode(x) for x in indexed_tokens]
tokens_tensor = torch.tensor([indexed_tokens]).to("cuda:1")
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]
predicted_top10 = torch.argsort(predictions[0][-1], descending=True)[:10]
predicted_text = [tokenizer.decode(x) for x in predicted_top10]
print(test_sentence, "=>", predicted_text)
# print(predictions)
# print(len(outputs))
# print(outputs)
all_pos_layer_input, all_pos_attn_output, all_pos_residual_output, all_pos_ffn_output, all_pos_layer_output, all_last_attn_subvalues, all_pos_coefficient_scores, all_attn_scores = transfer_output(outputs[1])
final_var = ((torch.var(torch.tensor(all_pos_layer_output[-1][-1]), -1, unbiased=False)+1e-5)**0.5).item()
pos_len = len(tokens)
print(tokens)

In [ ]:
predict_index = predicted_top10[0].item()
print(predict_index, tokenizer.decode(predict_index))

In [ ]:
#layer-level increase
all_attn_log_increase = []
for layer_i in range(LAYER_NUM):
    cur_attn_vector = torch.tensor(all_pos_attn_output[layer_i][-1])
    cur_layer_input = torch.tensor(all_pos_layer_input[layer_i][-1])
    origin_prob_log = torch.log(get_prob(get_bsvalues(cur_layer_input, model, final_var))[predict_index])
    cur_attn_vector_plus = cur_attn_vector + cur_layer_input
    cur_attn_vector_bsvalues = get_bsvalues(cur_attn_vector_plus, model, final_var)
    cur_attn_vector_probs = get_prob(cur_attn_vector_bsvalues)
    cur_attn_vector_probs = cur_attn_vector_probs[predict_index]
    cur_attn_vector_probs_log = torch.log(cur_attn_vector_probs)
    cur_attn_vector_probs_log_increase = cur_attn_vector_probs_log - origin_prob_log
    all_attn_log_increase.append(cur_attn_vector_probs_log_increase.item())
all_ffn_log_increase = []
for layer_i in range(LAYER_NUM):
    cur_ffn_vector = torch.tensor(all_pos_ffn_output[layer_i][-1])
    cur_residual = torch.tensor(all_pos_residual_output[layer_i][-1])
    origin_prob_log = torch.log(get_prob(get_bsvalues(cur_residual, model, final_var))[predict_index])
    cur_ffn_vector_plus = cur_ffn_vector + cur_residual
    cur_ffn_vector_bsvalues = get_bsvalues(cur_ffn_vector_plus, model, final_var)
    cur_ffn_vector_probs = get_prob(cur_ffn_vector_bsvalues)
    cur_ffn_vector_probs = cur_ffn_vector_probs[predict_index]
    cur_ffn_vector_probs_log = torch.log(cur_ffn_vector_probs)
    cur_ffn_vector_probs_log_increase = cur_ffn_vector_probs_log - origin_prob_log
    all_ffn_log_increase.append(cur_ffn_vector_probs_log_increase.tolist())
attn_list, ffn_list = [], []
for layer_i in range(LAYER_NUM):
    attn_list.append([str(layer_i), all_attn_log_increase[layer_i]])
    ffn_list.append([str(layer_i), all_ffn_log_increase[layer_i]])
attn_list_sort = sorted(attn_list, key=lambda x: x[-1])[::-1]#[:10]
ffn_list_sort = sorted(ffn_list, key=lambda x: x[-1])[::-1]#[:10]
attn_increase_compute, ffn_increase_compute = [], []
for indx, increase in attn_list_sort:
    attn_increase_compute.append((indx, round(increase, 3)))
for indx, increase in ffn_list_sort:
    ffn_increase_compute.append((indx, round(increase, 3)))
print("attn sum: ", sum([x[1] for x in attn_increase_compute]), 
      "ffn sum: ", sum([x[1] for x in ffn_increase_compute]))
print("attn: ", attn_increase_compute)
print("ffn: ", ffn_increase_compute)
all_increases_draw = []
for i in range(len(attn_list)):
    all_increases_draw.append(attn_list[i][1])
    all_increases_draw.append(ffn_list[i][1])    
plt_bar(range(len(all_increases_draw)), all_increases_draw)

In [ ]:
#head-level increase
all_head_increase = []
for test_layer in range(LAYER_NUM):
    cur_layer_input = torch.tensor(all_pos_layer_input[test_layer])
    cur_v_heads = torch.tensor(all_last_attn_subvalues[test_layer])
    cur_attn_o_split = model.transformer.h[test_layer].attn.out_proj.weight.data.view(HEAD_NUM, HEAD_DIM, -1)
    cur_attn_subvalues_headrecompute = torch.bmm(cur_v_heads, cur_attn_o_split).permute(1, 0, 2)
    cur_attn_subvalues_head_sum = torch.sum(cur_attn_subvalues_headrecompute, 0)
    cur_layer_input_last = cur_layer_input[-1]
    origin_prob = torch.log(get_prob(get_bsvalues(cur_layer_input_last, model, final_var))[predict_index])
    cur_attn_subvalues_head_plus = cur_attn_subvalues_head_sum + cur_layer_input_last
    cur_attn_plus_probs = torch.log(get_prob(get_bsvalues(
            cur_attn_subvalues_head_plus, model, final_var))[:, predict_index])
    cur_attn_plus_probs_increase = cur_attn_plus_probs - origin_prob
    for i in range(len(cur_attn_plus_probs_increase)):
        all_head_increase.append([str(test_layer)+"_"+str(i), round(cur_attn_plus_probs_increase[i].item(), 4)])

all_head_increase_sort = sorted(all_head_increase, key=lambda x:x[-1])[::-1]
print(all_head_increase_sort[:20])
all_head_increase_list = [x[1] for x in all_head_increase]
all_head_increase_list_split = torch.tensor(all_head_increase_list).view((LAYER_NUM, HEAD_NUM)).permute((1,0)).tolist()
plt_heatmap(all_head_increase_list_split)

In [ ]:
all_ffn_subvalues = []
for layer_i in range(LAYER_NUM):
    coefficient_scores = torch.tensor(all_pos_coefficient_scores[layer_i][-1])
    fc2_vectors = get_fc2_params(model, layer_i)
    ffn_subvalues = (coefficient_scores * fc2_vectors).T
    all_ffn_subvalues.append(ffn_subvalues)
ffn_subvalue_list = []
for layer_i in range(LAYER_NUM):
    cur_ffn_subvalues = all_ffn_subvalues[layer_i]
    cur_residual = torch.tensor(all_pos_residual_output[layer_i][-1])
    origin_prob_log = torch.log(get_prob(get_bsvalues(cur_residual, model, final_var))[predict_index])
    cur_ffn_subvalues_plus = cur_ffn_subvalues + cur_residual
    cur_ffn_subvalues_bsvalues = get_bsvalues(cur_ffn_subvalues_plus, model, final_var)
    cur_ffn_subvalues_probs = get_prob(cur_ffn_subvalues_bsvalues)
    cur_ffn_subvalues_probs = cur_ffn_subvalues_probs[:, predict_index]
    cur_ffn_subvalues_probs_log = torch.log(cur_ffn_subvalues_probs)
    cur_ffn_subvalues_probs_log_increase = cur_ffn_subvalues_probs_log - origin_prob_log
    for index, ffn_increase in enumerate(cur_ffn_subvalues_probs_log_increase):
        ffn_subvalue_list.append([str(layer_i)+"_"+str(index), ffn_increase.item()])
ffn_subvalue_list_sort = sorted(ffn_subvalue_list, key=lambda x: x[-1])[::-1]
for x in ffn_subvalue_list_sort[:10]:
    print(x[0], round(x[1], 4))
    layer = int(x[0].split("_")[0])
    neuron = int(x[0].split("_")[1])
    cur_vector = get_fc2_params(model, layer).T[neuron]
    cur_vector_bsvalue = get_bsvalues(cur_vector, model, final_var)
    cur_vector_bsvalue_sort = torch.argsort(cur_vector_bsvalue, descending=True)
    print("top10: ", [tokenizer.decode(a) for a in cur_vector_bsvalue_sort[:10]])
    print("last10: ", [tokenizer.decode(a) for a in cur_vector_bsvalue_sort[-10:]])

In [12]:
#attn neuron (pos)
cur_file_attn_neuron_list = []
for test_layer in range(LAYER_NUM):
    cur_layer_input = torch.tensor(all_pos_layer_input[test_layer])
    cur_v_heads_recompute = torch.tensor(all_last_attn_subvalues[test_layer]).permute(1, 0, 2)
    cur_attn_o_split = model.transformer.h[test_layer].attn.out_proj.weight.data.view(HEAD_NUM, HEAD_DIM, -1)
    cur_attn_o_recompute = cur_attn_o_split * cur_v_heads_recompute.unsqueeze(-1)
    cur_layer_input_last = cur_layer_input[-1]
    origin_prob = torch.log(get_prob(get_bsvalues(cur_layer_input_last, model, final_var))[predict_index])
    cur_attn_o_head_plus = cur_attn_o_recompute + cur_layer_input_last
    cur_attn_plus_probs = torch.log(get_prob(get_bsvalues(
        cur_attn_o_head_plus, model, final_var))[:, :, :, predict_index])
    cur_attn_plus_probs_increase = cur_attn_plus_probs - origin_prob
    for pos_index in range(cur_attn_plus_probs_increase.size(0)):
        for head_index in range(cur_attn_plus_probs_increase.size(1)):
            for attn_neuron_index in range(cur_attn_plus_probs_increase.size(2)):
                cur_file_attn_neuron_list.append((str(test_layer)+"_"+str(head_index)+"_"+str(
                    attn_neuron_index)+"_"+str(pos_index), 
                    cur_attn_plus_probs_increase[pos_index][head_index][attn_neuron_index].item()))
cur_file_attn_neuron_list_sort = sorted(cur_file_attn_neuron_list, key=lambda x: x[-1])[::-1]
print(list(zip(range(len(tokens)), tokens)))
for x in cur_file_attn_neuron_list_sort[:10]:
    layer_i, head_i, neuron_i, _ = x[0].split("_")
    layer_i, head_i, neuron_i = int(layer_i), int(head_i), int(neuron_i)
    cur_neuron = model.transformer.h[layer_i].attn.out_proj.weight.data.view(HEAD_NUM, HEAD_DIM, -1)[head_i][neuron_i]
    cur_neuron_bsvalue = get_bsvalues(cur_neuron, model, final_var)
    cur_neuron_bsvalue_sort = torch.argsort(cur_neuron_bsvalue, descending=True)
    print(x[0], round(x[1], 4), "top10: ", [tokenizer.decode(a) for a in cur_neuron_bsvalue_sort[:10]])
    print(x[0], round(x[1], 4), "last10: ", [tokenizer.decode(a) for a in cur_neuron_bsvalue_sort[-10:]])

In [20]:
#attn neuron (pos)
cur_file_attn_neuron_list = []
for test_layer in range(LAYER_NUM):
    cur_layer_input = torch.tensor(all_pos_layer_input[test_layer])
    cur_v_heads_recompute = torch.tensor(all_last_attn_subvalues[test_layer]).permute(1, 0, 2)
    cur_attn_o_split = model.transformer.h[test_layer].attn.c_proj.weight.data.view(HEAD_NUM, HEAD_DIM, -1)
    cur_attn_o_recompute = cur_attn_o_split * cur_v_heads_recompute.unsqueeze(-1)
    cur_layer_input_last = cur_layer_input[-1]
    origin_prob = torch.log(get_prob(get_bsvalues(cur_layer_input_last, model, final_var))[predict_index])
    cur_attn_o_head_plus = cur_attn_o_recompute + cur_layer_input_last
    cur_attn_plus_probs = torch.log(get_prob(get_bsvalues(
        cur_attn_o_head_plus, model, final_var))[:, :, :, predict_index])
    cur_attn_plus_probs_increase = cur_attn_plus_probs - origin_prob
    for pos_index in range(cur_attn_plus_probs_increase.size(0)):
        for head_index in range(cur_attn_plus_probs_increase.size(1)):
            for attn_neuron_index in range(cur_attn_plus_probs_increase.size(2)):
                cur_file_attn_neuron_list.append((str(test_layer)+"_"+str(head_index)+"_"+str(
                    attn_neuron_index)+"_"+str(pos_index), 
                    cur_attn_plus_probs_increase[pos_index][head_index][attn_neuron_index].item()))
cur_file_attn_neuron_list_sort = sorted(cur_file_attn_neuron_list, key=lambda x: x[-1])[::-1]
print(list(zip(range(len(tokens)), tokens)))
for x in cur_file_attn_neuron_list_sort[:10]:
    layer_i, head_i, neuron_i, _ = x[0].split("_")
    layer_i, head_i, neuron_i = int(layer_i), int(head_i), int(neuron_i)
    cur_neuron = model.transformer.h[layer_i].attn.c_proj.weight.data.view(HEAD_NUM, HEAD_DIM, -1)[head_i][neuron_i]
    cur_neuron_bsvalue = get_bsvalues(cur_neuron, model, final_var)
    cur_neuron_bsvalue_sort = torch.argsort(cur_neuron_bsvalue, descending=True)
    print(x[0], round(x[1], 4), "top10: ", [tokenizer.decode(a) for a in cur_neuron_bsvalue_sort[:10]])
    print(x[0], round(x[1], 4), "last10: ", [tokenizer.decode(a) for a in cur_neuron_bsvalue_sort[-10:]])

In [19]:
#find query FFN neurons activating attn neurons
curfile_ffn_score_dict = {}
for l_h_n_p, increase_score in cur_file_attn_neuron_list_sort[:30]:
    attn_layer, attn_head, attn_neuron, attn_pos = l_h_n_p.split("_")
    attn_layer, attn_head, attn_neuron, attn_pos = int(attn_layer), int(attn_head), int(attn_neuron), int(attn_pos)
    cur_attn_neuron = attn_head * HEAD_DIM + attn_neuron
    
    attn_neuron_key = model.transformer.h[attn_layer].attn.k_proj.weight.data[:, cur_attn_neuron]
    
    attn_neuron_key_new = attn_neuron_key * model.transformer.h[attn_layer].ln_1.weight.data
    cur_inner_all = torch.sum(torch.tensor(all_pos_layer_input[attn_layer][attn_pos]) * attn_neuron_key_new, -1)
    for layer_i in range(attn_layer):
        cur_layer_neurons = (torch.tensor(all_pos_coefficient_scores[layer_i][attn_pos])*get_fc2_params(model, layer_i)).T
        cur_layer_neurons_innerproduct = torch.sum(cur_layer_neurons * attn_neuron_key_new, -1)/cur_inner_all
        for neuron_i in range(len(cur_layer_neurons_innerproduct)):
            if str(layer_i)+"_"+str(neuron_i) not in curfile_ffn_score_dict:
                curfile_ffn_score_dict[str(layer_i)+"_"+str(neuron_i)] = 0.0
            curfile_ffn_score_dict[str(layer_i)+"_"+str(neuron_i)] += cur_layer_neurons_innerproduct[neuron_i].item() * increase_score
cur_file_neurons_ffn_zip = list(zip(curfile_ffn_score_dict.keys(), curfile_ffn_score_dict.values()))
cur_file_neurons_ffn_zip_sort = sorted(cur_file_neurons_ffn_zip, key=lambda x: x[-1])[::-1]
for x in cur_file_neurons_ffn_zip_sort[:10]:
    print(x[0], round(x[1], 4))
    layer = int(x[0].split("_")[0])
    neuron = int(x[0].split("_")[1])
    cur_vector = get_fc2_params(model, layer).T[neuron]
    cur_vector_bsvalue = get_bsvalues(cur_vector, model, final_var)
    cur_vector_bsvalue_sort = torch.argsort(cur_vector_bsvalue, descending=True)
    print("top10: ", [tokenizer.decode(a) for a in cur_vector_bsvalue_sort[:10]])
    print("last10: ", [tokenizer.decode(a) for a in cur_vector_bsvalue_sort[-10:]])